In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
import os
from tqdm.notebook import trange, tqdm
import numpy as np

In [2]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPVisionModel

In [3]:
import ssl
from irods.session import iRODSSession

In [4]:
from tools import batch

In [5]:
scratch_dir = "/scratch/leuven/355/vsc35521/"
mango_data_folder = "/ghum/home/ghum_pilot034/TEAM_3/"

In [6]:
try:
    env_file = os.environ['IRODS_ENVIRONMENT_FILE']
except KeyError:
    env_file = os.path.expanduser('~/.irods/irods_environment.json')

ssl_context = ssl.create_default_context(
        purpose=ssl.Purpose.SERVER_AUTH,
        cafile=None, capath=None, cadata=None
        )
ssl_settings = {'ssl_context': ssl_context}

In [7]:
session = iRODSSession(irods_env_file=env_file, **ssl_settings)
collection = session.collections.get("/ghum/home/ghum_pilot034/DATASET_3")

## Paths

In [8]:
df = pd.read_csv('dataset_3.csv')

In [9]:
df['mango_path'] = "/ghum/home/ghum_pilot034/DATASET_3/" + df['IE PID'] + "/" + df['REP PID'] + "/" + df['FL PID'] + "_" + df['NAME']

In [10]:
df['mango_output_path'] = "/ghum/home/ghum_pilot034/TEAM_3/vectors/" + df['IE PID'] + "_" + df['REP PID'] + "_" + df['FL PID'] + "_" + df['NAME']

In [11]:
df['server_path'] = scratch_dir + "DATASET_3/" + df['IE PID'] + "/" + df['REP PID'] + "/" + df['FL PID'] + "_" + df['NAME']

## Run CV model on the images

In [ ]:
model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [13]:
def get_CLIP_representation(pahts):
    
    for b_path in batch(paths, n=10):
        inputs = processor(images=(Image.open(path) in b_path), return_tensors="pt")

        outputs = model(**inputs)
        # last_hidden_state = outputs.last_hidden_state
        vectors = outputs.pooler_output
        break 

In [15]:
df['exists_vector'] = df['server_path'].apply(lambda x: os.path.exists(x + '.npy'))

In [21]:
paths = df[['server_path', 'mango_output_path']][~df['exists_vector']].values

In [ ]:
for code, batch_path in enumerate(tqdm(batch(paths, n=50))):

    inputs = processor(images=[Image.open(path[0])for path in batch_path], return_tensors="pt")
    outputs = model(**inputs)
    vectors = outputs.pooler_output.detach().numpy()
        
    for vector, (s_path, m_path) in zip(vectors, batch_path):
        vector_path = s_path + '.npy'
        np.save(vector_path, vector, allow_pickle=True, fix_imports=True)
        session.data_objects.put(vector_path, m_path + '.npy')

0it [00:00, ?it/s]

In [ ]:
paths.values